# Convolutional Neural Networks: MNIST Deep Dive

**Autor:** Samuel Leonardo Albarracín Vergara  
**Curso:** AREP  
**Fecha:** 03/02/2026

---

## Objetivo
Este notebook implementa y analiza una red neuronal convolucional (CNN) desde cero usando MNIST como dataset.
Se comparan arquitecturas con y sin convoluciones, se realizan experimentos controlados, y se interpretan las decisiones arquitectónicas.

# Parte 1: Exploración de Datos (EDA)

## 1.1 Carga del Dataset
Utilizamos MNIST desde TensorFlow Keras, que contiene:
- **60,000 imágenes de entrenamiento**
- **10,000 imágenes de prueba**
- Dígitos manuscritos (0-9)
- Dimensiones: 28×28 píxeles en escala de grises

In [ ]:
from tensorflow.keras.datasets import mnist
import matplotlib.pyplot as plt
import numpy as np

# Cargar datos
(x_train, y_train), (x_test, y_test) = mnist.load_data()

print(f"Tamaño de x_train: {x_train.shape}")
print(f"Tamaño de y_train: {y_train.shape}")
print(f"Tamaño de x_test: {x_test.shape}")
print(f"Tamaño de y_test: {y_test.shape}")
print(f"\nRango de píxeles: [{x_train.min()}, {x_train.max()}]")

## 1.2 Análisis de Distribución de Clases

In [ ]:
import pandas as pd

# Contar ejemplos por clase
unique_train, counts_train = np.unique(y_train, return_counts=True)
unique_test, counts_test = np.unique(y_test, return_counts=True)

# Crear tabla
dist_df = pd.DataFrame({
    'Dígito': unique_train,
    'Entrenamiento': counts_train,
    'Prueba': counts_test
})

print("Distribución de clases:")
print(dist_df)
print(f"\nTotal entrenamiento: {counts_train.sum()}")
print(f"Total prueba: {counts_test.sum()}")

**Observación:** Las clases están distribuidas de manera relativamente balanceada (aproximadamente 6000 muestras de entrenamiento por dígito).

## 1.3 Visualización de Ejemplos
Mostramos un ejemplo de cada dígito para entender la variabilidad del dataset.

In [ ]:
# Un ejemplo por clase
plt.figure(figsize=(12, 2))
for digit in range(10):
    idx = np.where(y_train == digit)[0][0]
    plt.subplot(1, 10, digit + 1)
    plt.imshow(x_train[idx], cmap='gray')
    plt.title(f"{digit}")
    plt.axis('off')
plt.tight_layout()
plt.suptitle('Un ejemplo de cada dígito', y=1.05)
plt.show()

## 1.4 Análisis de Píxeles y Preprocesamiento

Analizamos características estadísticas de los píxeles antes del preprocesamiento.

In [ ]:
# Estadísticas de píxeles
print("Estadísticas de píxeles (valor original 0-255):")
print(f"Min: {x_train.min()}, Max: {x_train.max()}")
print(f"Media: {x_train.mean():.2f}")
print(f"Desv. Est.: {x_train.std():.2f}")
print(f"Mediana: {np.median(x_train):.2f}")

# Histograma de valores
plt.figure(figsize=(10, 4))
plt.hist(x_train.flatten(), bins=50, edgecolor='black', alpha=0.7)
plt.xlabel('Intensidad de píxel')
plt.ylabel('Frecuencia')
plt.title('Distribución de intensidades de píxeles (Entrenamiento)')
plt.grid(alpha=0.3)
plt.show()

## 1.5 Normalización

Las redes neuronales entrenan mejor con valores normalizados en el rango [0, 1].
Dividimos por 255 para convertir píxeles del rango [0, 255] al rango [0, 1].

$$x_{normalizado} = \frac{x_{original}}{255}$$

In [ ]:
# Normalizar
x_train_norm = x_train / 255.0
x_test_norm = x_test / 255.0

print("Después de normalización:")
print(f"Min: {x_train_norm.min()}, Max: {x_train_norm.max()}")
print(f"Media: {x_train_norm.mean():.4f}")
print(f"Desv. Est.: {x_train_norm.std():.4f}")

## 1.6 Resumen EDA

| Característica | Valor |
|---|---|
| Imágenes de entrenamiento | 60,000 |
| Imágenes de prueba | 10,000 |
| Dimensiones de imagen | 28 × 28 píxeles |
| Canales | 1 (escala de grises) |
| Número de clases | 10 (dígitos 0-9) |
| Distribuición de clases | Balanceada (~6000 por clase) |
| Rango de píxeles original | [0, 255] |
| Rango normalizado | [0, 1] |

**Conclusión:** MNIST es un dataset ideal para CNN porque:
- La información espacial es crucial (píxeles cercanos están correlacionados)
- Las convoluciones pueden detectar características locales (bordes, esquinas)
- El dataset es pequeño (~1.4 GB en memoria) y se procesa rápidamente

# Parte 2: Modelo Baseline (Sin Convoluciones)

Antes de construir una CNN, establecemos un punto de referencia con una red **fully connected**.
Esto nos permite cuantificar cuánto mejora (o no) la CNN frente a un modelo que ignora
la estructura espacial de la imagen.

## Arquitectura del Baseline

```
Input (28x28) → Flatten (784) → Dense(128, ReLU) → Dense(10, Softmax)
```

- **Flatten**: convierte la imagen 2D en un vector 1D de 784 valores. Se pierde toda noción
  de qué píxeles son vecinos.
- **Dense(128, ReLU)**: capa oculta con 128 neuronas. Cada neurona recibe los 784 píxeles
  como entrada independiente (784×128 = 100,352 pesos + 128 biases).
- **Dense(10, Softmax)**: capa de salida que produce una distribución de probabilidad
  sobre las 10 clases.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.utils import to_categorical

# One-hot encoding de las etiquetas
y_train_oh = to_categorical(y_train, 10)
y_test_oh = to_categorical(y_test, 10)

# Construir el modelo baseline
baseline_model = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

baseline_model.summary()

## 2.1 Compilación y Entrenamiento

Usamos:
- **Optimizador Adam**: adapta la tasa de aprendizaje por parámetro, converge rápido en la mayoría de casos.
- **Loss: categorical crossentropy**: función de pérdida estándar para clasificación multiclase con one-hot encoding.
- **validation_split=0.1**: reservamos el 10% del entrenamiento para validación. Esto nos permite detectar overfitting (si la accuracy de entrenamiento sube pero la de validación se estanca o baja).

In [ ]:
baseline_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

baseline_history = baseline_model.fit(
    x_train_norm, y_train_oh,
    epochs=10,
    batch_size=32,
    validation_split=0.1,
    verbose=1
)

## 2.2 Curvas de Entrenamiento

Graficamos la evolución del loss y la accuracy a lo largo de los epochs, tanto para
entrenamiento como para validación. Esto permite observar si el modelo está aprendiendo
correctamente o si presenta overfitting.

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

# Loss
ax1.plot(baseline_history.history['loss'], label='Train')
ax1.plot(baseline_history.history['val_loss'], label='Validation')
ax1.set_title('Baseline — Loss por epoch')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.legend()
ax1.grid(alpha=0.3)

# Accuracy
ax2.plot(baseline_history.history['accuracy'], label='Train')
ax2.plot(baseline_history.history['val_accuracy'], label='Validation')
ax2.set_title('Baseline — Accuracy por epoch')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy')
ax2.legend()
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.show()

## 2.3 Evaluación en Test

In [ ]:
baseline_loss, baseline_acc = baseline_model.evaluate(x_test_norm, y_test_oh, verbose=0)

print(f"Baseline — Test Loss:     {baseline_loss:.4f}")
print(f"Baseline — Test Accuracy: {baseline_acc:.4f}")
print(f"\nTotal de parámetros: {baseline_model.count_params():,}")

## 2.4 Limitaciones del Modelo Baseline

El modelo fully connected tiene limitaciones fundamentales para datos de imagen:

1. **Ignora la estructura espacial:** `Flatten` destruye la relación 2D entre píxeles.
   El modelo no sabe que el píxel (5,5) es vecino del píxel (5,6). Trata una imagen
   igual que cualquier vector de 784 números sin orden.

2. **Alto número de parámetros:** Solo la primera capa Dense tiene 784×128 = 100,352 pesos.
   Cada neurona se conecta a *todos* los píxeles, incluso a los que no aportan información
   (los bordes negros, por ejemplo). Esto es ineficiente.

3. **Sin invariancia a traslación:** Si un dígito "3" aparece ligeramente desplazado,
   el modelo lo ve como un patrón completamente diferente porque los mismos píxeles
   activan neuronas distintas.

4. **No comparte características:** Si el modelo aprende a detectar un borde vertical
   en la esquina superior izquierda, no puede reutilizar ese conocimiento para detectar
   el mismo borde en otra posición de la imagen.

Estas limitaciones son exactamente las que las **capas convolucionales** resuelven mediante
weight sharing y receptive fields locales.